In [1]:

from skimage.measure import block_reduce

from framework.dependency import *
from framework.utli import *

In [3]:
def pipeline(X, Qstep=16, Q_mode=1, ML_inv=True, write=False, name='tmp.txt', tPCA=None, isDCT=True):
    H, W = X.shape[0], X.shape[1]
    X = X.reshape(1, H, W, -1)
    X_p, X_q, X_r = X[:,:,:,0:1], block_reduce(X[:,:,:,1:2], (1, 2, 2, 1), np.mean), block_reduce(X[:,:,:,2:], (1, 2, 2, 1), np.mean)
    # P
    def proP(X_p, Qstep, Q_mode, isDCT):
        X_block = Shrink(X_p, win=8)
        if isDCT == False:
            trans_pca = myPCA(is2D=True, H=8, W=8)
            trans_pca.fit(X_block)
        else:
            trans_pca = DCT(8,8)
        tX = trans_pca.transform(X_block)
        tX = Q(trans_pca, tX, Qstep, mode=Q_mode)
        return trans_pca, tX
        # Quant
    def inv_proP(trans_pca, tX, Qstep, Xraw, ML_inv, Q_mode):
        Xraw = Shrink(Xraw, win=8)
        tX = dQ(trans_pca, tX, Qstep, mode=Q_mode)
        if ML_inv == True:
            iX = trans_pca.ML_inverse_transform(Xraw, tX)
        else:
            iX = trans_pca.inverse_transform(tX)
        iX_p = invShrink(iX, win=8)
        return iX_p

    # QR
    def proQR(trans_pca, X_q, Qstep, Q_mode):
        X_block = Shrink(X_q, win=8)
        tX = trans_pca.transform(X_block)
        tX = Q(trans_pca, tX, Qstep, mode=Q_mode)
        return tX
        
    if tPCA == None:
        tPCA, tX_p = proP(X_p, Qstep, Q_mode, isDCT)
    else:
        tX_p = proQR(tPCA, X_p, Qstep, Q_mode)
    if write == True:
        write_to_txt(tX_p, name)
    iX_p = inv_proP(tPCA, tX_p, Qstep, X_p, ML_inv, Q_mode)

    tX_q = proQR(tPCA, X_q, Qstep, Q_mode)
    if write == True:
        write_to_txt(tX_q, name)
    iX_q = inv_proP(tPCA, tX_q, Qstep, X_q, ML_inv, Q_mode)
    iX_q = cv2.resize(iX_q[0,:,:,0], (W, H)).reshape(1, H, W, 1)

    tX_r = proQR(tPCA, X_r, Qstep, Q_mode)
    if write == True:
        write_to_txt(tX_r, name)
        with open(name, 'a') as f:
            f.write('-1111')
    iX_r = inv_proP(tPCA, tX_r, Qstep, X_r, ML_inv, Q_mode)
    iX_r = cv2.resize(iX_r[0,:,:,0], (W, H)).reshape(1, H, W, 1)
    return np.concatenate((iX_p, iX_q, iX_r), axis=-1)
    

def run(tPCA=None, ML_inv=True, ML_color_inv=True, img=0, write=False, isYUV=True, name=None, Q_mode=0, isDCT=False):
    psnr = []
    if isDCT == True:
        q = np.arange(5, 99, 5)
    else:
        q = [200, 160, 140, 120, 100, 90, 80, 70, 60, 50, 40.0,32.0,26.6,22.8,20.0,17.7,16.0,14.4,12.8,11.2,9.6,8.0,6.4,4.8,3.2]
    q = np.arange(5, 99, 5)
    for i in range(len(q)):     
        X_bgr = cv2.imread('/Users/alex/Desktop/proj/compression/data/Kodak/Kodak/'+str(img)+'.bmp')
        if isYUV == False:
            color_pca, X = BGR2PQR(X_bgr)
        else:
            X = BGR2YUV(X_bgr)
        iX = pipeline(X, Qstep=q[i], Q_mode=Q_mode, ML_inv=ML_inv, write=write, name='../result/'+name+'/'+str(img)+'_'+str(i)+'.txt', tPCA=tPCA, isDCT=isDCT)
        if ML_color_inv == True:
            iX = ML_inv_color(X_bgr, iX)
            #cv2.imwrite(str(img)+'_'+str(i)+'.png', copy.deepcopy(iX))
            psnr.append(PSNR(iX, X_bgr))
        else:
            if isYUV == True:
                iX = YUV2BGR(iX)
            else:
                iX = PQR2BGR(iX, color_pca)
            psnr.append(PSNR(iX[0], X_bgr))
        #break
    return psnr
  
    
psnr = []
name = 'tmp'
for i in range(24):
    psnr.append(run(None, 1, 1, img=i, write=1, isYUV=False, name=name, Q_mode=3, isDCT=False))
    #break
psnr = np.array(psnr)
with open('../result/psnr_'+name+'.pkl', 'wb') as f:
    pickle.dump(psnr,f)

write  393216 to ../result/tmp/0_0.txt 3 -9
write  98304 to ../result/tmp/0_0.txt 1 -3
write  98304 to ../result/tmp/0_0.txt 0 0
write  393216 to ../result/tmp/0_1.txt 5 -18
write  98304 to ../result/tmp/0_1.txt 2 -7
write  98304 to ../result/tmp/0_1.txt 1 -1
write  393216 to ../result/tmp/0_2.txt 8 -26
write  98304 to ../result/tmp/0_2.txt 3 -10
write  98304 to ../result/tmp/0_2.txt 1 -1
write  393216 to ../result/tmp/0_3.txt 10 -35
write  98304 to ../result/tmp/0_3.txt 4 -13
write  98304 to ../result/tmp/0_3.txt 2 -2
write  393216 to ../result/tmp/0_4.txt 13 -44
write  98304 to ../result/tmp/0_4.txt 4 -17
write  98304 to ../result/tmp/0_4.txt 2 -2
write  393216 to ../result/tmp/0_5.txt 15 -53
write  98304 to ../result/tmp/0_5.txt 5 -20
write  98304 to ../result/tmp/0_5.txt 3 -2
write  393216 to ../result/tmp/0_6.txt 18 -62
write  98304 to ../result/tmp/0_6.txt 6 -23
write  98304 to ../result/tmp/0_6.txt 3 -3
write  393216 to ../result/tmp/0_7.txt 20 -70
write  98304 to ../result/tmp/

write  98304 to ../result/tmp/3_3.txt 11 -5
write  98304 to ../result/tmp/3_3.txt 7 -2
write  393216 to ../result/tmp/3_4.txt 55 -18
write  98304 to ../result/tmp/3_4.txt 14 -6
write  98304 to ../result/tmp/3_4.txt 9 -2
write  393216 to ../result/tmp/3_5.txt 65 -22
write  98304 to ../result/tmp/3_5.txt 17 -7
write  98304 to ../result/tmp/3_5.txt 10 -3
write  393216 to ../result/tmp/3_6.txt 76 -26
write  98304 to ../result/tmp/3_6.txt 20 -9
write  98304 to ../result/tmp/3_6.txt 12 -3
write  393216 to ../result/tmp/3_7.txt 87 -29
write  98304 to ../result/tmp/3_7.txt 23 -10
write  98304 to ../result/tmp/3_7.txt 14 -3
write  393216 to ../result/tmp/3_8.txt 98 -33
write  98304 to ../result/tmp/3_8.txt 25 -11
write  98304 to ../result/tmp/3_8.txt 15 -4
write  393216 to ../result/tmp/3_9.txt 109 -37
write  98304 to ../result/tmp/3_9.txt 28 -12
write  98304 to ../result/tmp/3_9.txt 17 -4
write  393216 to ../result/tmp/3_10.txt 121 -41
write  98304 to ../result/tmp/3_10.txt 31 -14
write  98304

write  98304 to ../result/tmp/6_6.txt 3 -4
write  393216 to ../result/tmp/6_7.txt 28 -87
write  98304 to ../result/tmp/6_7.txt 5 -12
write  98304 to ../result/tmp/6_7.txt 3 -5
write  393216 to ../result/tmp/6_8.txt 32 -98
write  98304 to ../result/tmp/6_8.txt 5 -14
write  98304 to ../result/tmp/6_8.txt 3 -6
write  393216 to ../result/tmp/6_9.txt 35 -108
write  98304 to ../result/tmp/6_9.txt 6 -15
write  98304 to ../result/tmp/6_9.txt 4 -6
write  393216 to ../result/tmp/6_10.txt 39 -121
write  98304 to ../result/tmp/6_10.txt 7 -17
write  98304 to ../result/tmp/6_10.txt 4 -7
write  393216 to ../result/tmp/6_11.txt 44 -136
write  98304 to ../result/tmp/6_11.txt 8 -19
write  98304 to ../result/tmp/6_11.txt 5 -8
write  393216 to ../result/tmp/6_12.txt 51 -155
write  98304 to ../result/tmp/6_12.txt 9 -21
write  98304 to ../result/tmp/6_12.txt 5 -9
write  393216 to ../result/tmp/6_13.txt 59 -181
write  98304 to ../result/tmp/6_13.txt 10 -25
write  98304 to ../result/tmp/6_13.txt 6 -10
write  

write  393216 to ../result/tmp/9_10.txt 94 -24
write  98304 to ../result/tmp/9_10.txt 27 -9
write  98304 to ../result/tmp/9_10.txt 1 -2
write  393216 to ../result/tmp/9_11.txt 106 -26
write  98304 to ../result/tmp/9_11.txt 30 -10
write  98304 to ../result/tmp/9_11.txt 2 -3
write  393216 to ../result/tmp/9_12.txt 121 -30
write  98304 to ../result/tmp/9_12.txt 35 -11
write  98304 to ../result/tmp/9_12.txt 2 -3
write  393216 to ../result/tmp/9_13.txt 141 -35
write  98304 to ../result/tmp/9_13.txt 40 -13
write  98304 to ../result/tmp/9_13.txt 2 -4
write  393216 to ../result/tmp/9_14.txt 169 -42
write  98304 to ../result/tmp/9_14.txt 48 -15
write  98304 to ../result/tmp/9_14.txt 2 -4
write  393216 to ../result/tmp/9_15.txt 211 -53
write  98304 to ../result/tmp/9_15.txt 60 -19
write  98304 to ../result/tmp/9_15.txt 3 -5
write  393216 to ../result/tmp/9_16.txt 282 -71
write  98304 to ../result/tmp/9_16.txt 81 -26
write  98304 to ../result/tmp/9_16.txt 4 -7
write  393216 to ../result/tmp/9_17.

write  98304 to ../result/tmp/12_12.txt 39 -30
write  98304 to ../result/tmp/12_12.txt 11 -30
write  393216 to ../result/tmp/12_13.txt 42 -162
write  98304 to ../result/tmp/12_13.txt 45 -35
write  98304 to ../result/tmp/12_13.txt 13 -35
write  393216 to ../result/tmp/12_14.txt 50 -195
write  98304 to ../result/tmp/12_14.txt 54 -42
write  98304 to ../result/tmp/12_14.txt 16 -42
write  393216 to ../result/tmp/12_15.txt 63 -244
write  98304 to ../result/tmp/12_15.txt 68 -53
write  98304 to ../result/tmp/12_15.txt 20 -53
write  393216 to ../result/tmp/12_16.txt 84 -325
write  98304 to ../result/tmp/12_16.txt 91 -70
write  98304 to ../result/tmp/12_16.txt 27 -71
write  393216 to ../result/tmp/12_17.txt 126 -487
write  98304 to ../result/tmp/12_17.txt 136 -106
write  98304 to ../result/tmp/12_17.txt 40 -106
write  393216 to ../result/tmp/12_18.txt 251 -975
write  98304 to ../result/tmp/12_18.txt 272 -211
write  98304 to ../result/tmp/12_18.txt 80 -212
write  393216 to ../result/tmp/13_0.txt 

write  98304 to ../result/tmp/15_14.txt 12 -30
write  98304 to ../result/tmp/15_14.txt 10 -12
write  393216 to ../result/tmp/15_15.txt 78 -274
write  98304 to ../result/tmp/15_15.txt 15 -37
write  98304 to ../result/tmp/15_15.txt 13 -16
write  393216 to ../result/tmp/15_16.txt 104 -365
write  98304 to ../result/tmp/15_16.txt 20 -49
write  98304 to ../result/tmp/15_16.txt 17 -21
write  393216 to ../result/tmp/15_17.txt 155 -547
write  98304 to ../result/tmp/15_17.txt 30 -74
write  98304 to ../result/tmp/15_17.txt 26 -31
write  393216 to ../result/tmp/15_18.txt 311 -1095
write  98304 to ../result/tmp/15_18.txt 60 -148
write  98304 to ../result/tmp/15_18.txt 51 -62
write  393216 to ../result/tmp/16_0.txt 3 -10
write  98304 to ../result/tmp/16_0.txt 2 -1
write  98304 to ../result/tmp/16_0.txt 1 -1
write  393216 to ../result/tmp/16_1.txt 6 -21
write  98304 to ../result/tmp/16_1.txt 4 -3
write  98304 to ../result/tmp/16_1.txt 2 -1
write  393216 to ../result/tmp/16_2.txt 9 -31
write  98304 to

write  98304 to ../result/tmp/18_16.txt 125 -40
write  98304 to ../result/tmp/18_16.txt 28 -36
write  393216 to ../result/tmp/18_17.txt 490 -111
write  98304 to ../result/tmp/18_17.txt 187 -60
write  98304 to ../result/tmp/18_17.txt 42 -54
write  393216 to ../result/tmp/18_18.txt 981 -222
write  98304 to ../result/tmp/18_18.txt 374 -120
write  98304 to ../result/tmp/18_18.txt 85 -107
write  393216 to ../result/tmp/19_0.txt 11 -3
write  98304 to ../result/tmp/19_0.txt 5 -2
write  98304 to ../result/tmp/19_0.txt 2 -1
write  393216 to ../result/tmp/19_1.txt 22 -6
write  98304 to ../result/tmp/19_1.txt 9 -4
write  98304 to ../result/tmp/19_1.txt 4 -3
write  393216 to ../result/tmp/19_2.txt 33 -9
write  98304 to ../result/tmp/19_2.txt 14 -6
write  98304 to ../result/tmp/19_2.txt 6 -4
write  393216 to ../result/tmp/19_3.txt 44 -13
write  98304 to ../result/tmp/19_3.txt 19 -8
write  98304 to ../result/tmp/19_3.txt 8 -6
write  393216 to ../result/tmp/19_4.txt 55 -16
write  98304 to ../result/t

write  393216 to ../result/tmp/21_18.txt 330 -1065
write  98304 to ../result/tmp/21_18.txt 94 -227
write  98304 to ../result/tmp/21_18.txt 28 -76
write  393216 to ../result/tmp/22_0.txt 4 -11
write  98304 to ../result/tmp/22_0.txt 4 -2
write  98304 to ../result/tmp/22_0.txt 1 0
write  393216 to ../result/tmp/22_1.txt 7 -22
write  98304 to ../result/tmp/22_1.txt 9 -3
write  98304 to ../result/tmp/22_1.txt 1 -1
write  393216 to ../result/tmp/22_2.txt 11 -33
write  98304 to ../result/tmp/22_2.txt 13 -5
write  98304 to ../result/tmp/22_2.txt 2 -1
write  393216 to ../result/tmp/22_3.txt 14 -44
write  98304 to ../result/tmp/22_3.txt 17 -6
write  98304 to ../result/tmp/22_3.txt 2 -1
write  393216 to ../result/tmp/22_4.txt 18 -55
write  98304 to ../result/tmp/22_4.txt 22 -8
write  98304 to ../result/tmp/22_4.txt 3 -1
write  393216 to ../result/tmp/22_5.txt 21 -66
write  98304 to ../result/tmp/22_5.txt 26 -10
write  98304 to ../result/tmp/22_5.txt 3 -2
write  393216 to ../result/tmp/22_6.txt 25